### Thinking1
ALS都有哪些应用场景

答：ALS可以应用在电影推荐（基于评分等显示行为数据的ALS），商品推荐、广告推送（基于浏览、点击等隐式行为数据的ALS）。

### Thinking2
ALS进行矩阵分解的时候，为什么可以并行化处理

答：由于ALS每次固定一个变量，更新另一个，如固定$Y$，$X$的更新表达式为$$ X_i = (YY^T + \lambda I)^{-1} YR_i$$&emsp;&emsp;其中$X_i$为用户$i$的向量，为k维列向量，$Y=[Y_1,Y_2...Y_M]$为商品矩阵，$R_i$为用户$i$评分向量，为m维列向量（评分矩阵：$R=[R_1,R_2...R_N]^T$）  
&emsp;&emsp;也就是说，用户矩阵$X=[X_1,X_2...X_N]$中每一个列向量的更新之和$Y$与$R$中对应的一行有关，相互之间是独立的，所以可以并行化处理。

### Thinking3
梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）

答：批量梯度下降是每次都使用全体样本来对参数进行更新，随机梯度下降是每次随机选取一个样本对参数进行更新，小批量梯度下降是每次随机选取部分样本来对参数进行更新。

### Thinking4
你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中

答：Restricted Boltzmann Machines for Collaborative Filtering——将受限的玻尔兹曼机用在协同过滤上，一个用户作为一个样本，将其每一个评分平铺成向量，形成评分矩阵输入，每个样本只影响有评分处的参数，模型训练完成后，并非是采用隐藏层得到的输出来计算相似度，而是直接利用编码再解码重构得到的评分矩阵来计算未评分处的评分。

### Action1
对MovieLens数据集进行评分预测
工具：可以使用Surprise或者其他
说明使用的模型，及简要原理

使用模型：BaselineModel  
原理：将所有电影的均分$\mu$加上用户对整体偏差$b_u$加上商品对整体的偏差$b_i$，得到预测值$b=\mu+b_u+b_i$，然后对评分减预测值的平方和再加上正则化项进行优化，优化方法可用交替最小二乘法（ALS)，也可用随机梯度下降法（SGD）

In [100]:
import pandas as pd
from surprise import Reader

# 数据加载
pd_data = pd.read_csv('./ratings.csv')
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)
#也可以load_from_df
train_set = data.build_full_trainset()

# 数据探索（查询user 0)
print(pd_data.head())
train_set.ur[0]

   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580


[(0, 3.5),
 (1, 3.5),
 (2, 3.5),
 (3, 3.5),
 (4, 3.5),
 (5, 3.5),
 (6, 4.0),
 (7, 4.0),
 (8, 4.0),
 (9, 4.0),
 (10, 4.0),
 (11, 4.0),
 (12, 4.0),
 (13, 3.5),
 (14, 3.5),
 (15, 4.0),
 (16, 3.5),
 (17, 3.5),
 (18, 3.0),
 (19, 3.5),
 (20, 3.5),
 (21, 3.5),
 (22, 4.0),
 (23, 4.0),
 (24, 3.5),
 (25, 3.5),
 (26, 4.0),
 (27, 4.0),
 (28, 3.5),
 (29, 3.5),
 (30, 4.5),
 (31, 4.5),
 (32, 4.0),
 (33, 3.0),
 (34, 3.5),
 (35, 4.0),
 (36, 4.0),
 (37, 3.5),
 (38, 4.0),
 (39, 3.5),
 (40, 4.0),
 (41, 3.0),
 (42, 3.5),
 (43, 4.0),
 (44, 4.0),
 (45, 4.0),
 (46, 3.5),
 (47, 3.5),
 (48, 4.0),
 (49, 4.0),
 (50, 3.5),
 (51, 3.0),
 (52, 4.0),
 (53, 4.0),
 (54, 3.5),
 (55, 3.5),
 (56, 4.0),
 (57, 3.0),
 (58, 4.0),
 (59, 4.0),
 (60, 3.0),
 (61, 3.5),
 (62, 3.5),
 (63, 3.5),
 (64, 3.5),
 (65, 4.0),
 (66, 3.5),
 (67, 3.5),
 (68, 4.0),
 (69, 4.0),
 (70, 4.0),
 (71, 4.0),
 (72, 4.0),
 (73, 4.0),
 (74, 4.0),
 (75, 4.0),
 (76, 4.0),
 (77, 3.5),
 (78, 3.5),
 (79, 4.0),
 (80, 4.0),
 (81, 4.0),
 (82, 4.0),
 (83, 3.5),
 (

In [44]:
from surprise import BaselineOnly
from surprise.model_selection import KFold
from surprise import accuracy
import time

# ALS
als = {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}
#SGD
sgd = {'method': 'sgd', 'n_epochs': 5}

algo_als = BaselineOnly(bsl_options=als)
algo_sgd = BaselineOnly(bsl_options=sgd)

# 定义K折交叉验证
def kfold(data, algo, k=3):
    kf = KFold(n_splits=k)
    sum_arr = 0
    start = time.time()
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        # 计算RMSE
        sum_arr += accuracy.rmse(predictions, verbose=True)
    end = time.time()
    return sum_arr/k, (end-start)/k

print('对于ALS：')
arr, t = kfold(data, algo_als)
print('k折交叉验证的平均准确率为：',arr)
print('平均用时为：',t)

print('对于SGD：')
arr, t = kfold(data, algo_sgd)
print('k折交叉验证的平均准确率为：',arr)
print('平均用时为：',t)

对于ALS：
Estimating biases using als...
RMSE: 0.8649
Estimating biases using als...
RMSE: 0.8632
Estimating biases using als...
RMSE: 0.8634
k折交叉验证的平均准确率为： 0.8638121264775914
平均用时为： 4.647355794906616
对于SGD：
Estimating biases using sgd...
RMSE: 0.8746
Estimating biases using sgd...
RMSE: 0.8731
Estimating biases using sgd...
RMSE: 0.8752
k折交叉验证的平均准确率为： 0.8742830718392947
平均用时为： 4.42904535929362


In [110]:
# 推荐topN
movies = list(pd_data['movieId'].drop_duplicates(keep=False))
def recommander(uid, trainset, movielist, algo, n=5):
    algo.fit(trainset)
    result = {}
    rated = [i[0] for i in train_set.ur[uid]]
    for iid in movielist:
        if iid in rated:
            continue
        pred = algo.predict(uid,iid)[3]
        result[iid] = pred
    return sorted(result.items(), key = lambda x: x[1], reverse=True)[:n]

print('对于User 0的top5推荐为：')
recommander(0, train_set, movies, algo_sgd)

对于User 0的top5推荐为：
Estimating biases using sgd...


[(60524, 3.5292716305462175),
 (70227, 3.5292716305462175),
 (59382, 3.5292716305462175),
 (60482, 3.5292716305462175),
 (2489, 3.5292716305462175)]

### Action2
Paper Reading：Slope one predictors for online rating-based collaborative filtering. Daniel Lemire and Anna Maclachlan, 2007. http://arxiv.org/abs/cs/0702144.
积累，总结笔记，自己的思考及idea

总结：  
&emsp;&emsp;文章介绍了基础的Slope one算法，以及两种改进的Slope one——有权重的（the weighted slope one）以及根据划分喜恶的（the bi-polar slope one），并与几个有代表性的算法进行了效果比较，说明了slope one算法在具有易于实现、更新迅速、查询高效、对用户历史数据要求少这几个优点的同时，仍然具有有竞争性的准确性。  

自己的思考：  
&emsp;&emsp;除了Slope one的想法本身，文章介绍的第二种优化更加体现了协同过滤的思想，基于用户评分的平均值，将用户评过分的电影分为“用户喜欢”和“用户不喜欢”，并且以用户U，查询电影i为例，假设j为用户U喜欢的电影，在计入j对i的偏差值的时，只考虑那些同样喜欢j并且还喜欢i的用户，共同喜欢j代表了用户之间的相似，j和i被同时喜欢代表了电影间的相似，这里充分地体现了协同过滤的思想。一开始我认为应该将“相似”用户都算进去，但仔细想过之后发现要求这些用户对i、j同喜恶是有道理的，因为当人和人喜欢的东西相同时并不代表他们讨厌的东西也相同。

### Action3
设计你自己的句子生成器  
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''

In [40]:
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''

In [41]:
# 生成语法树
def getGrammardict(grammar, linesplit='\n', gramsplit='=>'):
    result = {}
    for line in grammar.split(linesplit):
        if not line.strip():
            continue
        exp, statement = line.split(gramsplit)
        result[exp.strip()] = [i.split() for i in statement.split('|')]
    return result
grammardict = getGrammardict(grammar)
grammardict  

{'战斗': [['施法', '，', '结果', '。']],
 '施法': [['主语', '动作', '技能']],
 '结果': [['主语', '获得', '效果']],
 '主语': [['张飞'],
  ['关羽'],
  ['赵云'],
  ['典韦'],
  ['许褚'],
  ['刘备'],
  ['黄忠'],
  ['曹操'],
  ['鲁班七号'],
  ['貂蝉']],
 '动作': [['施放'], ['使用'], ['召唤']],
 '技能': [['一骑当千'],
  ['单刀赴会'],
  ['青龙偃月'],
  ['刀锋铁骑'],
  ['黑暗潜能'],
  ['画地为牢'],
  ['守护机关'],
  ['狂兽血性'],
  ['龙鸣'],
  ['惊雷之龙'],
  ['破云之龙'],
  ['天翔之龙']],
 '获得': [['损失'], ['获得']],
 '效果': [['数值', '状态']],
 '数值': [['1'], ['1000'], ['5000'], ['100']],
 '状态': [['法力'], ['生命']]}

In [46]:
# 生成句子
import random
def generate(grammardict, target, isEng=False):
    if target not in grammardict:
        return target
    find = random.choice(grammardict[target])
    blank =''
    if isEng:
        blank = ' '
    return blank.join(generate(grammardict, i, isEng) for i in find)
generate(grammardict, '战斗')
        

'黄忠召唤破云之龙，关羽损失1法力。'